In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
data = np.load('train.npy')

In [3]:
x_train = data[:, 0]
y_train = data[:, 1]

In [4]:
enc = LabelEncoder()
Y_train_t = enc.fit_transform(y_train)
Y_train = np_utils.to_categorical(Y_train_t, 500)
del y_train
del Y_train_t

In [5]:
for i in range(len(x_train)):
    x_train[i] = imresize(x_train[i], size=(56, 56))

In [6]:
x_train = np.dstack(x_train)
x_train=np.rollaxis(x_train, 2)
x_train = x_train.reshape(x_train.shape[0], 56, 56, 1)

In [7]:
X_train = x_train.astype('float16')
X_train = -X_train + 255
X_train /= 256

In [8]:
for i in range(len(X_train)):
    X_train[i] /= np.amax(X_train[i])

In [9]:
del x_train
del data

In [10]:
test_size = 1

In [11]:
x_train = X_train
y_train = Y_train
x_test = X_train[-test_size:]
y_test = Y_train[-test_size:]

In [12]:
print(x_train[0].shape)

(56, 56, 1)


In [13]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import PReLU
from keras import backend as K

In [20]:
batch_size = 256
nb_classes = 500
nb_epoch = 12

img_rows, img_cols = 56, 56
input_shape = (img_rows, img_cols, 1)
nb_filters = 64
pool_size = (2, 2)
kernel_size = (3, 3)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(PReLU(init='zero', weights=None))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(nb_filters * 2, kernel_size[0], kernel_size[1]))
model.add(PReLU(init='zero', weights=None))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(nb_filters * 4, kernel_size[0], kernel_size[1]))
model.add(PReLU(init='zero', weights=None))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(322))
model.add(PReLU(init='zero', weights=None))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 166707 samples, validate on 1 samples
Epoch 1/12
166707/166707 [==============================] - 2759s - loss: 3.5473 - acc: 0.2878 - val_loss: 1.5312 - val_acc: 1.0000

In [22]:
val_data = np.load('test.npy')
for i in range(len(val_data)):
    val_data[i] = imresize(val_data[i], size=(56, 56))
    
val_data = np.dstack(val_data)
val_data = np.rollaxis(val_data, 2)
val_data = val_data.reshape(val_data.shape[0], 56, 56, 1)

val_data_float = val_data.astype('float16')
val_data_float = -val_data_float + 255
val_data_float /= 256

for i in range(len(val_data_float)):
    val_data_float[i] /= np.amax(val_data_float[i])

In [28]:
val_classes = model.predict_classes(val_data_float)

41428/41428 [==============================] - 249s     

In [29]:
labels = enc.inverse_transform(val_classes)

In [30]:
out = open('out3.txt', 'w+')
out.write('Id,Category\n')
for i in range(len(labels)):
    out.write(str(i + 1) + ',' + str(labels[i]) + '\n')
out.close()

In [26]:
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=6,
          verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 166707 samples, validate on 1 samples
Epoch 1/6
166707/166707 [==============================] - 2671s - loss: 0.2170 - acc: 0.9338 - val_loss: 0.0051 - val_acc: 1.0000

In [27]:
del val_classes
del labels

In [32]:
from keras.models import load_model
model.save('my_mod1.h5')

In [33]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 54, 54, 64)    640         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
prelu_11 (PReLU)                 (None, 54, 54, 64)    186624      convolution2d_8[0][0]            
____________________________________________________________________________________________________
maxpooling2d_8 (MaxPooling2D)    (None, 27, 27, 64)    0           prelu_11[0][0]                   
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 25, 25, 128)   73856       maxpooling2d_8[0][0]             
___________________________________________________________________________________________

In [ ]:
batch_size = 256
nb_classes = 500
nb_epoch = 12

img_rows, img_cols = 56, 56
input_shape = (img_rows, img_cols, 1)
nb_filters = 64
pool_size = (2, 2)
kernel_size = (3, 3)

model2 = Sequential()

model2.add(Convolution2D(75, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model2.add(PReLU(init='zero', weights=None))
model2.add(MaxPooling2D(pool_size=pool_size))

model2.add(Convolution2D(150, kernel_size[0], kernel_size[1]))
model2.add(PReLU(init='zero', weights=None))
model2.add(MaxPooling2D(pool_size=pool_size))

model2.add(Convolution2D(300, 5, 5))
model2.add(PReLU(init='zero', weights=None))

model2.add(Flatten())


model2.add(Dropout(0.5))
model2.add(Dense(750))
model2.add(PReLU(init='zero', weights=None))
model2.add(Dropout(0.5))

model2.add(Dense(nb_classes))
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model2.fit(x_train, y_train, batch_size=300, nb_epoch=3,
          verbose=1, validation_data=(x_test, y_test))

Train on 166708 samples, validate on 1 samples
Epoch 1/3
  1500/166708 [..............................] - ETA: 4382s - loss: 6.2478 - acc: 6.6667e-04